# Confronto fra mio DeCUR state_dict e ICAfusion state_dict()

Nota importante: in "ADL-Project/DeCUR/src/pretrain/demo_load_weight.ipynb" c'è probabilmente una spiegazione molto chiara di come andrebbe caricati i pesi dentro un modello, forse si può fare riferimento anche a quello per quanto riguarda il codice definitivo (da implementare dentro icafusion(?)).

In questa prima cella, cerco di capire la differenza in termini di nomi fra i checkpoint che vengono salvati da 

In [10]:
import torch

# icafusion state_dict = torch.load("/mnt/proj3/eu-25-19/davide_secco/ADL-Project/ICAFusion/state_dict_from_ckpt.pth", map_location=torch.device('cpu'))

### Load DeCUR ckpt (.pth) and model ( .pt)
decur_original_state_dict_pth = torch.load("/mnt/proj3/eu-25-19/davide_secco/ADL-Project/DeCUR/checkpoint/KAIST_Training_forSSL_pretraining_25/checkpoint_0014.pth", map_location=torch.device('cpu'))
try:
    original_model_from_DeCUR_pt = torch.load("/mnt/proj3/eu-25-19/davide_secco/ADL-Project/DeCUR/checkpoint/KAIST_Training_forSSL_pretraining_25/checkpoint_0014.pt", map_location=torch.device('cpu'), weights_only=False)
except Exception as e:
    print(f"Error loading original_model_from_DeCUR_pt: {e}")
    print("Penso sia dovuto al fatto che il modello è stato salvato come oggetto python e non come state_dict, non ci serve")
    original_model_from_DeCUR_pt = None

print(f"Type of original_sate_dict_from_DeCUR_pth: {type(decur_original_state_dict_pth)}")
if isinstance(decur_original_state_dict_pth, dict):
    print("Keys in original_state_dict_from_DeCUR_pth:", decur_original_state_dict_pth.keys())

### Load DeCUR and ICAFusion MODEL state_dict (la differenza sta nel fatto che per DeCUR io avevo salvato altro oltre a model, invece per icasfusion solo il model)
decur_model_state_dict = decur_original_state_dict_pth['model']
icafusion_model_state_dict = torch.load("/mnt/proj3/eu-25-19/davide_secco/ADL-Project/ICAFusion/icafusion_state_dict_from_Model.pth", map_location=torch.device('cpu'))
densedecur_model_state_dict = torch.load("./DenseDeCUR/checkpoint/checkpoint_0003.pth", map_location="cpu", weights_only=False)['model']
offset_densedecur = 4

decur_model = list(decur_model_state_dict.items())
icafusion_model = list(icafusion_model_state_dict.items())
densedecur_model = list(densedecur_model_state_dict.items())

fine_compatibilità = 318

min_len = min(len(decur_model), len(icafusion_model), len(densedecur_model))

for index in range(0, fine_compatibilità):
    if isinstance(decur_model[index][1], torch.Tensor) and isinstance(icafusion_model[index][1], torch.Tensor):
        print(f"{index}: {decur_model[index][0]} {decur_model[index][1].shape} -> {decur_model[index][0]} {decur_model[index][1].shape} -> {densedecur_model[index+offset_densedecur][0]} {densedecur_model[index+offset_densedecur][1].shape}")

offset_densedecur =+ 12

for index in range(fine_compatibilità, min_len):
    # Only print shape if both are tensors
    if isinstance(decur_model[index][1], torch.Tensor) and isinstance(icafusion_model[index][1], torch.Tensor):
        print(f"{index}: {decur_model[index][0]} {decur_model[index][1].shape} -> {decur_model[index][0]} {decur_model[index][1].shape} -> {densedecur_model[index+offset_densedecur][0]} {densedecur_model[index+offset_densedecur][1].shape}")
        
if len(decur_model) > min_len:
    print("\nExtra keys in state_dict:")
    for index in range(min_len, len(decur_model)):
        print(f"{index}: {decur_model[index][0]} -> [no corresponding model key]")

if len(icafusion_model) > min_len:
    print("\nExtra keys in model_state_dict:")
    for index in range(min_len, len(icafusion_model)):
        print(f"{index}: [no corresponding state key] -> {icafusion_model[index][0]}")

# LAYER in più di densedecur
for index in range(0,4):
    if isinstance(decur_model[index][1], torch.Tensor) and isinstance(icafusion_model[index][1], torch.Tensor):
        print(f"{densedecur_model[index][0]} {densedecur_model[index][1].shape}")

print("-------")

for index in range(fine_compatibilità+4, fine_compatibilità+12):
    if isinstance(decur_model[index][1], torch.Tensor) and isinstance(icafusion_model[index][1], torch.Tensor):
        print(f"{densedecur_model[index][0]} {densedecur_model[index][1].shape}")


Error loading original_model_from_DeCUR_pt: No module named 'models'
Penso sia dovuto al fatto che il modello è stato salvato come oggetto python e non come state_dict, non ci serve
Type of original_sate_dict_from_DeCUR_pth: <class 'dict'>
Keys in original_state_dict_from_DeCUR_pth: dict_keys(['epoch', 'model', 'optimizer'])
0: module.backbone_1.conv1.weight torch.Size([64, 3, 7, 7]) -> module.backbone_1.conv1.weight torch.Size([64, 3, 7, 7]) -> module.encoder_q.0.conv1.weight torch.Size([64, 3, 7, 7])
1: module.backbone_1.bn1.weight torch.Size([64]) -> module.backbone_1.bn1.weight torch.Size([64]) -> module.encoder_q.0.bn1.weight torch.Size([64])
2: module.backbone_1.bn1.bias torch.Size([64]) -> module.backbone_1.bn1.bias torch.Size([64]) -> module.encoder_q.0.bn1.bias torch.Size([64])
3: module.backbone_1.bn1.running_mean torch.Size([64]) -> module.backbone_1.bn1.running_mean torch.Size([64]) -> module.encoder_q.0.bn1.running_mean torch.Size([64])
4: module.backbone_1.bn1.running_var

In [9]:
[x for x in densedecur_model_state_dict.keys() if x.startswith('module.encoder_q.0.layer1.0')]

['module.encoder_q.0.layer1.0.conv1.weight',
 'module.encoder_q.0.layer1.0.bn1.weight',
 'module.encoder_q.0.layer1.0.bn1.bias',
 'module.encoder_q.0.layer1.0.bn1.running_mean',
 'module.encoder_q.0.layer1.0.bn1.running_var',
 'module.encoder_q.0.layer1.0.bn1.num_batches_tracked',
 'module.encoder_q.0.layer1.0.conv2.weight',
 'module.encoder_q.0.layer1.0.bn2.weight',
 'module.encoder_q.0.layer1.0.bn2.bias',
 'module.encoder_q.0.layer1.0.bn2.running_mean',
 'module.encoder_q.0.layer1.0.bn2.running_var',
 'module.encoder_q.0.layer1.0.bn2.num_batches_tracked',
 'module.encoder_q.0.layer1.0.conv3.weight',
 'module.encoder_q.0.layer1.0.bn3.weight',
 'module.encoder_q.0.layer1.0.bn3.bias',
 'module.encoder_q.0.layer1.0.bn3.running_mean',
 'module.encoder_q.0.layer1.0.bn3.running_var',
 'module.encoder_q.0.layer1.0.bn3.num_batches_tracked',
 'module.encoder_q.0.layer1.0.downsample.0.weight',
 'module.encoder_q.0.layer1.0.downsample.1.weight',
 'module.encoder_q.0.layer1.0.downsample.1.bias',


#### Dall'analisi fatta sopra posso dire che il mapping:

In [2]:
# Parte comune fra le due architetture
for index in range(0,636):
    print(f"{index}: {decur_model[index][0]} {decur_model[index][1].shape}-> {icafusion_model[index][0]} {icafusion_model[index][1].shape}")

0: module.backbone_1.conv1.weight torch.Size([64, 3, 7, 7])-> model.0.layer.0.weight torch.Size([64, 3, 7, 7])
1: module.backbone_1.bn1.weight torch.Size([64])-> model.0.layer.1.weight torch.Size([64])
2: module.backbone_1.bn1.bias torch.Size([64])-> model.0.layer.1.bias torch.Size([64])
3: module.backbone_1.bn1.running_mean torch.Size([64])-> model.0.layer.1.running_mean torch.Size([64])
4: module.backbone_1.bn1.running_var torch.Size([64])-> model.0.layer.1.running_var torch.Size([64])
5: module.backbone_1.bn1.num_batches_tracked torch.Size([])-> model.0.layer.1.num_batches_tracked torch.Size([])
6: module.backbone_1.layer1.0.conv1.weight torch.Size([64, 64, 1, 1])-> model.1.layer.0.conv1.weight torch.Size([64, 64, 1, 1])
7: module.backbone_1.layer1.0.bn1.weight torch.Size([64])-> model.1.layer.0.bn1.weight torch.Size([64])
8: module.backbone_1.layer1.0.bn1.bias torch.Size([64])-> model.1.layer.0.bn1.bias torch.Size([64])
9: module.backbone_1.layer1.0.bn1.running_mean torch.Size([64]

In [3]:
# Parte differente fra le due architetture
print("\nParte differente fra le due architetture: HEAD di DeCUR")
for index in range(636, len(decur_model)):
    print(f"{index}: {decur_model[index][0]} {decur_model[index][1].shape}")


Parte differente fra le due architetture: HEAD di DeCUR
636: module.projector1.0.weight torch.Size([8192, 2048])
637: module.projector1.1.weight torch.Size([8192])
638: module.projector1.1.bias torch.Size([8192])
639: module.projector1.1.running_mean torch.Size([8192])
640: module.projector1.1.running_var torch.Size([8192])
641: module.projector1.1.num_batches_tracked torch.Size([])
642: module.projector1.3.weight torch.Size([8192, 8192])
643: module.projector1.4.weight torch.Size([8192])
644: module.projector1.4.bias torch.Size([8192])
645: module.projector1.4.running_mean torch.Size([8192])
646: module.projector1.4.running_var torch.Size([8192])
647: module.projector1.4.num_batches_tracked torch.Size([])
648: module.projector1.6.weight torch.Size([8192, 8192])
649: module.projector2.0.weight torch.Size([8192, 2048])
650: module.projector2.1.weight torch.Size([8192])
651: module.projector2.1.bias torch.Size([8192])
652: module.projector2.1.running_mean torch.Size([8192])
653: module.

In [4]:
print("\nParte differente fra le due architetture: HEAD di ICAfusion")
for index in range(636, len(icafusion_model)):
    print(f"{index}: {icafusion_model[index][0]} {icafusion_model[index][1].shape}")


Parte differente fra le due architetture: HEAD di ICAfusion
636: model.10.pos_emb_vis torch.Size([1, 400, 512])
637: model.10.pos_emb_ir torch.Size([1, 400, 512])
638: model.10.vis_coefficient.w1 torch.Size([1])
639: model.10.vis_coefficient.w2 torch.Size([1])
640: model.10.ir_coefficient.w1 torch.Size([1])
641: model.10.ir_coefficient.w2 torch.Size([1])
642: model.10.crosstransformer.0.ln_input.weight torch.Size([512])
643: model.10.crosstransformer.0.ln_input.bias torch.Size([512])
644: model.10.crosstransformer.0.ln_output.weight torch.Size([512])
645: model.10.crosstransformer.0.ln_output.bias torch.Size([512])
646: model.10.crosstransformer.0.crossatt.que_proj_vis.weight torch.Size([512, 512])
647: model.10.crosstransformer.0.crossatt.que_proj_vis.bias torch.Size([512])
648: model.10.crosstransformer.0.crossatt.key_proj_vis.weight torch.Size([512, 512])
649: model.10.crosstransformer.0.crossatt.key_proj_vis.bias torch.Size([512])
650: model.10.crosstransformer.0.crossatt.val_proj

#### Visualizzo Visivamente le liste chiavi dei layer dei rispettivi modelli

In [5]:
import torch
from models.decur import DeCUR
import torch.distributed 

print(f"Decur model keys:      {[key for key, value in decur_model]}")
print(f"Icafusion model keys:  {[key for key, value in icafusion_model]}")

Decur model keys:      ['module.backbone_1.conv1.weight', 'module.backbone_1.bn1.weight', 'module.backbone_1.bn1.bias', 'module.backbone_1.bn1.running_mean', 'module.backbone_1.bn1.running_var', 'module.backbone_1.bn1.num_batches_tracked', 'module.backbone_1.layer1.0.conv1.weight', 'module.backbone_1.layer1.0.bn1.weight', 'module.backbone_1.layer1.0.bn1.bias', 'module.backbone_1.layer1.0.bn1.running_mean', 'module.backbone_1.layer1.0.bn1.running_var', 'module.backbone_1.layer1.0.bn1.num_batches_tracked', 'module.backbone_1.layer1.0.conv2.weight', 'module.backbone_1.layer1.0.bn2.weight', 'module.backbone_1.layer1.0.bn2.bias', 'module.backbone_1.layer1.0.bn2.running_mean', 'module.backbone_1.layer1.0.bn2.running_var', 'module.backbone_1.layer1.0.bn2.num_batches_tracked', 'module.backbone_1.layer1.0.conv3.weight', 'module.backbone_1.layer1.0.bn3.weight', 'module.backbone_1.layer1.0.bn3.bias', 'module.backbone_1.layer1.0.bn3.running_mean', 'module.backbone_1.layer1.0.bn3.running_var', 'mod

In [ ]:
import torch
from collections import OrderedDict

out_path = "/mnt/proj3/eu-25-19/davide_secco/ADL-Project/ICAFusion/checkpoint_0014_REWIRED_first636.pth"

tgt_keys = [key for key, value in icafusion_model[:636]]    

# --- SANITY CHECK ---
if len(decur_model) < len(tgt_keys):
    raise RuntimeError(f"Pochi parametri sorgente: src={len(decur_model)} < tgt636={len(tgt_keys)}")

# --- COSTRUISCI NUOVO STATE_DICT DEL CHECKPOINT (CHIAVI=MODEL, VALORI=CKPT in ordine) ---
model_updated = OrderedDict()
mismatches = []   # (i, src_key, tgt_key, src_shape, tgt_shape)

for i, (key, values) in enumerate(icafusion_model[:636]):   
    if decur_model[i][1].shape == icafusion_model[i][1].shape:
        model_updated[key] = decur_model[i][1]
    else:
        mismatches.append((i, decur_model[i][0], icafusion_model[i][0], decur_model[i][1].shape, icafusion_model[i][1].shape))
        # puoi scegliere: o fallire, o saltare, o cercare il prossimo compatibile.
        # qui fallisco esplicitamente per evitare silent bugs:
        # raise ValueError(f"Shape mismatch @#{i}: {src_key} {src_shape} -> {tgt_key} {tgt_shape}")

print(f"model_updated keys: {[key for key, value in model_updated.items()]}")

# --- REPORT DI CONFRONTO ---
print("=== CONFRONTO ORDINATO (prime 10) ===")
for i in range(min(10, len(icafusion_model))):
    print(f"[{i:03d}] {decur_model[i][0]}  -->  {icafusion_model[i][0]}   "
          f"{tuple(decur_model[i][1].shape)} == {tuple(icafusion_model[i][1].shape)}")

print("\n=== RIEPILOGO ===")
print(f"Target chiavi usate (model[:636]): {len(tgt_keys)}")
print(f"Nuove voci assegnate:              {len(model_updated)}")
print(f"Mismatches di shape:               {len(mismatches)}")

if mismatches:
    print("\nEsempi di mismatch (max 10):")
    for m in mismatches[:10]:
        i, src_k, tgt_k, sshape, tshape = m
        print(f"  #{i}: {src_k} {sshape}  ->  {tgt_k} {tshape}")

# --- CREA NUOVO CHECKPOINT (stessa struttura dell’originale, ma 'model' sostituito) ---
new_checkpoint = dict(decur_original_state_dict_pth)
new_checkpoint["model"] = model_updated

torch.save(new_checkpoint, out_path)
print(f"\nSalvato nuovo checkpoint: {out_path}")

new_model keys: ['model.0.layer.0.weight', 'model.0.layer.1.weight', 'model.0.layer.1.bias', 'model.0.layer.1.running_mean', 'model.0.layer.1.running_var', 'model.0.layer.1.num_batches_tracked', 'model.1.layer.0.conv1.weight', 'model.1.layer.0.bn1.weight', 'model.1.layer.0.bn1.bias', 'model.1.layer.0.bn1.running_mean', 'model.1.layer.0.bn1.running_var', 'model.1.layer.0.bn1.num_batches_tracked', 'model.1.layer.0.conv2.weight', 'model.1.layer.0.bn2.weight', 'model.1.layer.0.bn2.bias', 'model.1.layer.0.bn2.running_mean', 'model.1.layer.0.bn2.running_var', 'model.1.layer.0.bn2.num_batches_tracked', 'model.1.layer.0.conv3.weight', 'model.1.layer.0.bn3.weight', 'model.1.layer.0.bn3.bias', 'model.1.layer.0.bn3.running_mean', 'model.1.layer.0.bn3.running_var', 'model.1.layer.0.bn3.num_batches_tracked', 'model.1.layer.0.shortcut.0.weight', 'model.1.layer.0.shortcut.1.weight', 'model.1.layer.0.shortcut.1.bias', 'model.1.layer.0.shortcut.1.running_mean', 'model.1.layer.0.shortcut.1.running_var',

#### Controllo finale - Ricarico il checkpoint e mi aspetto che abbia le chiavi di ICAfusione e i valori di DeCUR

In [9]:
# --- VERIFICA FINALE DI RICARICA ---
ck2 = torch.load(out_path, map_location="cpu", weights_only=False)
model_updated = list(ck2["model"].items())
print(type(model_updated))
print(f"model_updated[0]: {model_updated[0][0]} {model_updated[0][1].shape}")

# Verifico che le chiavi del model_updated in output siano identiche a quelle di icafusion
print("\n=== VERIFICA ===")
print("Numero chiavi in 'model' (nuovo):", len(model_updated))
print("Ordine identico di chiavi a model[:636]?    ", [key for key, values in model_updated] == [key for key, values in icafusion_model[:636]])
print(f"list(model_updated.keys()) {[key for key, values in model_updated]}")
print(f"list(icafusion_model_keys[:636]) {[key for key, value in icafusion_model[:636]]}")

# Confronto valori tra model_updated e decur (prime 10)
print("Values identiche a model[:636]? (prime 10):")
for i in range(10):
    v1 = model_updated[i][1]
    v2 = decur_model[i][1]

    if isinstance(v1, torch.Tensor) and isinstance(v2, torch.Tensor):
        eq = torch.equal(v1, v2)
    else:
        eq = v1 == v2

    print(f"[{i:03d}] {icafusion_model[i][0]}: {eq}")


<class 'list'>
model_updated[0]: model.0.layer.0.weight torch.Size([64, 3, 7, 7])

=== VERIFICA ===
Numero chiavi in 'model' (nuovo): 636
Ordine identico di chiavi a model[:636]?     True
list(model_updated.keys()) ['model.0.layer.0.weight', 'model.0.layer.1.weight', 'model.0.layer.1.bias', 'model.0.layer.1.running_mean', 'model.0.layer.1.running_var', 'model.0.layer.1.num_batches_tracked', 'model.1.layer.0.conv1.weight', 'model.1.layer.0.bn1.weight', 'model.1.layer.0.bn1.bias', 'model.1.layer.0.bn1.running_mean', 'model.1.layer.0.bn1.running_var', 'model.1.layer.0.bn1.num_batches_tracked', 'model.1.layer.0.conv2.weight', 'model.1.layer.0.bn2.weight', 'model.1.layer.0.bn2.bias', 'model.1.layer.0.bn2.running_mean', 'model.1.layer.0.bn2.running_var', 'model.1.layer.0.bn2.num_batches_tracked', 'model.1.layer.0.conv3.weight', 'model.1.layer.0.bn3.weight', 'model.1.layer.0.bn3.bias', 'model.1.layer.0.bn3.running_mean', 'model.1.layer.0.bn3.running_var', 'model.1.layer.0.bn3.num_batches_trac

#### controllo optimizer

In [ ]:
print(f"{checkpoint['optimizer']['state'].keys()}")
print(f"{checkpoint['optimizer']['param_groups']}")

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,